In [ ]:
using Pkg
Pkg.add(url = "https://github.com/MilesCranmer/SymbolicRegression.jl.git", rev = "support-extra-data")

In [1]:
using MLJ  # for fit/predict
using SymbolicRegression  # for SRRegressor
using Zygote  # For `enable_autodiff=true`
using SymbolicUtils
using NPZ
using Pandas

In [2]:
true_f(x) = x^3 / 3 - cos(x)
deriv_f(x) = x^2 + sin(x)

deriv_f (generic function with 1 method)

In [3]:
X = reshape(0.0:0.32:10.0, :, 1)
y = true_f.(X[:, 1])
∂y = deriv_f.(X[:, 1])

32-element Vector{Float64}:
  0.0
  0.41696656061611775
  1.006795441362392
  1.7407915683009982
  2.596415860289225
  3.5595736030415055
  4.626045473685325
  5.801915925084421
  7.102955436427127
  8.55301934966111
 10.181625856572422
 12.020959041455523
 14.102601257946091
  ⋮
 41.0765492048505
 45.58145539714299
 50.248209128707636
 55.050052009553035
 59.96730333407156
 64.98935824662338
 70.11576444366216
 75.35626809573589
 80.7298243269406
 86.26267271702045
 91.98567321877702
 97.93117297286523

In [4]:
function derivative_loss(tree, dataset::Dataset{T,L}, options, idx) where {T,L}
    # Select from the batch indices, if given
    X = idx === nothing ? dataset.X : view(dataset.X, :, idx)

    # Evaluate both f(x) and f'(x), where f is defined by `tree`
    ŷ, ∂ŷ, completed = eval_grad_tree_array(tree, X, options; variable=true)

    #println(size(dataset.extra.∂y))

    !completed && return L(Inf)

    y = idx === nothing ? dataset.y : view(dataset.y, idx)
    ∂y = idx === nothing ? dataset.extra.∂y : view(dataset.extra.∂y, idx)

    mse_deriv = sum(i -> (∂ŷ[i] - ∂y[i])^2, eachindex(∂y)) / length(∂y)
    mse_value = sum(i -> (ŷ[i] - y[i])^2, eachindex(y)) / length(y)

    return mse_value  + mse_deriv
end

derivative_loss (generic function with 1 method)

In [5]:
model = SRRegressor(;
    binary_operators=[+, -, *],
    unary_operators=[cos],
    loss_function=derivative_loss,
    enable_autodiff=true,
    batching=true,
    batch_size=25,
    niterations=100,
    early_stop_condition=1e-6,
)
mach = machine(model, X, y, (; ∂y=∂y))
fit!(mach)

┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc SymbolicRegression.SRRegressor` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{AbstractMatrix{Continuous}, AbstractVector{Continuous}, Table{AbstractVector{Continuous}}}
│ 
│ fit_data_scitype(model) = Union{Tuple{Union{Table{<:AbstractVector{<:Continuous}}, AbstractMatrix{<:Continuous}}, AbstractVector{<:Continuous}}, Tuple{Union{Table{<:AbstractVector{

Started!


0.0%┣                                           ┫ 0/1.5k [00:01<-25:-34, -1s/it]Expressions evaluated per second: [.....]. Head worker occupation: 0.0%         Press 'q' and then <enter> to stop execution early.                             Hall of Fame:                                                                   ---------------------------------------------------------------------------------------------------                                                             Complexity  Loss       Score     Equation                                       1           1.707e+04  3.604e+01  y = x₁                                        3           8.230e+03  3.648e-01  y = (x₁ * x₁)                                 5           2.329e+03  6.312e-01  y = ((x₁ + x₁) * x₁)                          7           7.564e+02  5.623e-01  y = (((x₁ + x₁) + x₁) * x₁)                   9           6.000e+02  1.158e-01  y = (((x₁ + x₁) + x₁) * (x₁ - 0.84207))       10          5.965e+02  5.925e-03  y = ((

trained Machine; caches model-specific representations of data
  model: SRRegressor(binary_operators = Function[+, -, *], …)
  args: 
    1:	Source @058 ⏎ AbstractMatrix{Continuous}
    2:	Source @330 ⏎ AbstractVector{Continuous}
    3:	Source @179 ⏎ Table{AbstractVector{Continuous}}


In [ ]:
# load in the eta data

In [13]:
# load in numpy data

skip = 10

jacobian = npzread("mean_jacobian_for_sr.npy")

X = npzread("toy_data_for_sr.npy")[1:skip:10000, 1:6]

#y = npzread("theta_eta_for_sr_model_0.npy")[1:skip:10000, 7]


#∂y = jacobian[1:skip:10000, 1, :] # learn the first element of the jacobian row 
# this will be the integral wrt A

#size(X), size(y), size(∂y)

((1000, 6), (1000,), (1000, 6))

In [9]:
model = SRRegressor(;
    binary_operators=[+, *, ^],
    unary_operators=[log],
    constraints=[(^)=>(-1, 9)],
    nested_constraints=[(^) => [(^) => 0, log => 0],
                   log => [(^) =>  0, log => 0],
            #        exp => [log => 0]
        ],
    loss_function=derivative_loss,
    enable_autodiff=true,
    batching=false,
    #batch_size=100,
    niterations=100,
    parsimony=100,
)
mach = machine(model, X, y, (; ∂y=∂y))
fit!(mach)

┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc SymbolicRegression.SRRegressor` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{AbstractMatrix{Continuous}, AbstractVector{Continuous}, Unknown}
│ 
│ fit_data_scitype(model) = Union{Tuple{Union{Table{<:AbstractVector{<:Continuous}}, AbstractMatrix{<:Continuous}}, AbstractVector{<:Continuous}}, Tuple{Union{Table{<:AbstractVector{<:Continuous}}, AbstractMa

Started!


0.0%┣                                           ┫ 0/1.5k [00:01<-18:-57, -1s/it]Expressions evaluated per second: [.....]. Head worker occupation: 0.0%         Press 'q' and then <enter> to stop execution early.                             Hall of Fame:                                                                   ---------------------------------------------------------------------------------------------------                                                             Complexity  Loss       Score     Equation                                       1           1.600e+04  1.594e+01  y = x₁                                        3           5.881e+03  5.004e-01  y = (-12.901 * -8.3245)                       ---------------------------------------------------------------------------------------------------
0.1%┣                                           ┫ 2/1.5k [00:03<01:03:09, 3s/it]Expressions evaluated per second: 4.29e+02. Head worker occupation: 0.0%        Press 'q' and then <

LoadError: TaskFailedException

[91m    nested task error: [39mTaskFailedException
    Stacktrace:
     [1] [0m[1mwait[22m
    [90m   @ [39m[90m./[39m[90m[4mtask.jl:345[24m[39m[90m [inlined][39m
     [2] [0m[1mfetch[22m
    [90m   @ [39m[90m./[39m[90m[4mtask.jl:360[24m[39m[90m [inlined][39m
     [3] [0m[1m(::SymbolicRegression.var"#53#80"{Vector{Vector{Channel{Any}}}, Vector{Vector{Task}}, Int64, Int64})[22m[0m[1m([22m[0m[1m)[22m
    [90m   @ [39m[35mSymbolicRegression[39m [90m./[39m[90m[4mtask.jl:484[24m[39m
    
    [91m    nested task error: [39mInterruptException:
        Stacktrace:
          [1] [0m[1mPullback[22m
        [90m    @ [39m[90m~/.julia/packages/SymbolicRegression/fHd3u/src/[39m[90m[4mOperators.jl:47[24m[39m[90m [inlined][39m
          [2] [0m[1m(::Zygote.Pullback{Tuple{typeof(safe_pow), Float32, Float32}, Any})[22m[0m[1m([22m[90mΔ[39m::[0mFloat32[0m[1m)[22m
        [90m    @ [39m[35mZygote[

In [ ]:
r = report(mach)
eq = r.equations[r.best_idx]

variable_names = ["A_", "B_", "C_", "D_", "E_", "F_"]
symbolic_eq = node_to_symbolic(eq, model, variable_names=variable_names)

## loss for fitting $y=\partial y$
here we want to pass the derivatives as an extra and fit the candidate expression to that

In [3]:
function old_derivative_loss(tree, dataset::Dataset{T,L}, options, idx) where {T,L}
    # Column-major:
    X = idx === nothing ? dataset.X : view(dataset.X, :, idx)
    ∂y = idx === nothing ? dataset.y : view(dataset.y, idx)

    ŷ, ∂ŷ, completed = eval_grad_tree_array(tree, X, options; variable=true)

    !completed && return L(Inf)

    mse = sum(i -> (∂ŷ[i] - ∂y[i])^2, eachindex(∂y)) / length(∂y)
    return mse
end

old_derivative_loss (generic function with 1 method)

In [4]:
function fit_derivative_loss(tree, dataset::Dataset{T,L}, options, idx) where {T,L}
    # Column-major:
    X = idx === nothing ? dataset.X : view(dataset.X, :, idx)
    #∂y = idx === nothing ? dataset.y : view(dataset.y, idx)

    ŷ, ∂ŷ, completed = eval_grad_tree_array(tree, X, options; variable=true)

    !completed && return L(Inf)

    y = idx === nothing ? dataset.y : view(dataset.y, idx)
    ∂y = idx === nothing ? dataset.extra.∂y : view(dataset.extra.∂y, idx)
    
    # match the derivative only
    mse = sum(i -> (∂ŷ[i] - ∂y[i])^2, eachindex(∂y)) / length(∂y)
    
    return mse
end

fit_derivative_loss (generic function with 1 method)

In [5]:
# load in numpy data

skip = 10

jacobian = npzread("mean_jacobian_for_sr.npy")

X = npzread("theta_eta_for_sr_model_0.npy")[1:skip:10000, 1:6]

y = npzread("theta_eta_for_sr_model_0.npy")[1:skip:10000, 7]


∂y = jacobian[1:skip:10000, 1, :] # learn the first element of the jacobian row 
# this will be the integral wrt A

size(X), size(y), size(∂y)

((1000, 6), (1000,), (1000, 6))

In [ ]:
model = SRRegressor(;
    binary_operators=[+, *, ^],
    unary_operators=[log],
    constraints=[(^)=>(-1, 9)],
    nested_constraints=[(^) => [(^) => 0, log => 0],
                   log => [(^) =>  0, log => 0],
            #        exp => [log => 0]
        ],
    loss_function=fit_derivative_loss,
    enable_autodiff=true,
    batching=true,
    batch_size=100,
    niterations=100,
    parsimony=100,
)
mach = machine(model, X, y, (; ∂y=∂y))
fit!(mach)

In [ ]:
r = report(mach)
eq = r.equations[r.best_idx]

variable_names = ["A_", "B_", "C_", "D_", "E_", "F_"]
symbolic_eq = node_to_symbolic(eq, model, variable_names=variable_names)